In [ ]:
import tensorflow as tf
from tensorflow import keras
import pandas as pd
import numpy as np

from tensorflow.keras import optimizers, losses, activations, models
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, LearningRateScheduler, ReduceLROnPlateau
from tensorflow.keras.layers import Dense, Input, Dropout, Convolution1D, MaxPool1D, GlobalMaxPool1D, GlobalAveragePooling1D, \
    concatenate
from sklearn.metrics import f1_score, accuracy_score
from tensorflow.keras import layers
from tensorflow.keras.utils import to_categorical

In [ ]:
dt_train = pd.read_csv("/content/drive/My Drive/ecg/mitbih_train.csv", header=None)
dt_train = dt_train.sample(frac=1)
dt_test = pd.read_csv("/content/drive/My Drive/ecg/mitbih_test.csv", header=None)

train_y = np.array(dt_train[187].values).astype(np.int8)
train_x = np.array(dt_train[list(range(187))].values)[..., np.newaxis]

test_y = np.array(dt_test[187].values).astype(np.int8)
test_x = np.array(dt_test[list(range(187))].values)[..., np.newaxis]

In [ ]:
train_y = to_categorical(train_y)
train_y.shape
test_y = to_categorical(test_y)
test_y.shape

In [ ]:
model = tf.keras.Sequential()

model.add(layers.Conv1D(32, kernel_size=5,strides=1, padding="valid",input_shape=[187,1],activation='tanh'))
model.add(layers.BatchNormalization())
model.add(layers.Dropout(0.3))

model.add(layers.Conv1D(32, kernel_size=5,strides=2,  padding='valid',activation='tanh'))
model.add(layers.BatchNormalization())
model.add(layers.Dropout(0.3))

model.add(layers.Conv1D(64, kernel_size=5,strides=2, padding='valid',activation='tanh'))
model.add(layers.BatchNormalization())
model.add(layers.Dropout(0.3))

model.add(layers.Conv1D(128, kernel_size=5,strides=2, padding='valid',activation='tanh'))
model.add(layers.BatchNormalization())
model.add(layers.Dropout(0.3))

model.add(layers.Conv1D(256, kernel_size=5,strides=2, padding='valid',activation='tanh'))
model.add(layers.BatchNormalization())
model.add(layers.Dropout(0.3))

model.add(tf.keras.layers.Flatten())

model.add(tf.keras.layers.Dense(256,activation='tanh'))
model.add(layers.BatchNormalization())
model.add(tf.keras.layers.Dropout(0.2))

model.add(tf.keras.layers.Dense(5,activation='tanh'))
model.add(layers.Softmax())

In [ ]:
file_path = "/content/drive/My Drive/baseline_cnn_mitbih.h5"
checkpoint = ModelCheckpoint(file_path, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
early = EarlyStopping(monitor="val_acc", mode="max", patience=5, verbose=1)
redonplat = ReduceLROnPlateau(monitor="val_acc", mode="max", patience=3, verbose=2)
callbacks_list = [checkpoint, early, redonplat]  # early

In [ ]:
opt = tf.keras.optimizers.Adam(learning_rate=0.01)
model.compile(optimizer=opt, loss=losses.categorical_crossentropy, metrics=['acc'])
model.fit(train_x, train_y, epochs=1000,batch_size=10000, validation_split=0.1)

In [ ]:
model.load_weights(file_path)

In [ ]:
pred_test = model.predict(test_x)
pred_test = np.argmax(pred_test, axis=-1)

f1 = f1_score(test_y, pred_test, average="macro")

print("Test f1 score : %s "% f1)

acc = accuracy_score(test_y, pred_test)

print("Test accuracy score : %s "% acc)